##### Copyright 2019 The TensorFlow Authors.

In [ ]:
##### Copyright 2019 The TensorFlow Authors.

#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Keras를 이용한 마스킹 및 패딩 

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/keras/masking_and_padding">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    TensorFlow.org에서 보기</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/ko/guide/keras/masking_and_padding.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    구글 코랩(Colab)에서 실행하기</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/ko/guide/keras/masking_and_padding.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    깃허브(GitHub)에서 소스 보기</a>
  </td>
  <td>
    <a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/keras/masking_and_padding.ipynb">
    <img src="https://www.tensorflow.org/images/download_logo_32px.png" />
    노트북 다운로드</a>
  </td>
</table>

Note: 이 문서는 텐서플로 커뮤니티에서 번역했습니다. 커뮤니티 번역 활동의 특성상 정확한 번역과 최신 내용을 반영하기 위해 노력함에도 불구하고 [공식 영문 문서](https://github.com/tensorflow/docs/blob/master/site/en/guide/keras/masking_and_padding.ipynb)의 내용과 일치하지 않을 수 있습니다. 이 번역에 개선할 부분이 있다면 [tensorflow/docs](https://github.com/tensorflow/docs) 깃허브 저장소로 풀 리퀘스트를 보내주시기 바랍니다. 문서 번역이나 리뷰에 참여하려면 [docs-ko@tensorflow.org](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ko)로 메일을 보내주시기 바랍니다.

## 설정

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow.keras import layers

## 일련의 데이터 패딩

일련의 데이터를 처리 할 때 각각의 샘플 길이가 다른 것이 매우 일반적입니다. 다음 예제를 고려하십시오(단어로 토큰화 된 텍스트):

```
[
  ["The", "weather", "will", "be", "nice", "tomorrow"],
  ["How", "are", "you", "doing", "today"],
  ["Hello", "world", "!"]
]
```

어휘 조회 후, 데이터는 정수로 벡터화될. 수 있습니다, e.g.:

```
[
  [83, 91, 1, 645, 1253, 927],
  [73, 8, 3215, 55, 927],
  [71, 1331, 4231]
]
```

데이터는 개별 샘플의 길이가 각각 6, 5 및 3 인 2D 목록입니다. 딥러닝 모델의 입력 데이터는 단일 텐서 (이 경우 예: `(batch_size, 6, vocab_size)`)여야하므로 가장 긴 항목 보다 짧은 샘플에는 자리 표시자 값으로 채워져야 합니다 (또는 짧은 샘플을 채우기 전에 긴 샘플을 자를 수 도 있습니다).

Keras는 `tf.keras.preprocessing.sequence.pad_sequences`와 같이 시퀀스를 쉽게 잘라내어 일반적인 길이로 채울 수 있는 API를 제공합니다.

In [0]:
raw_inputs = [
  [83, 91, 1, 645, 1253, 927],
  [73, 8, 3215, 55, 927],
  [711, 632, 71]
]

# 기본적으로 0을 사용하여 채웁니다;
# "value" 매개 변수를 통해 구성할 수 있습니다.
# "pre" 패딩 (앞 부분에) 또는
# "post" 패딩 (끝 부분에)을 할 수 있습니다.
# RNN 레이어 작업 시 "post" 패딩을 사용하는 것을 추천합니다
# (레이어의 CuDNN 구현에 사용하기 위해).
# 
padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(raw_inputs,
                                                              padding='post')

print(padded_inputs)

## 마스킹

이제 모든 샘플의 길이가 균일하므로 데이터의 일부가 실제로 채워져있다는 사실을 모델에 알려야합니다. 그 메커니즘이 <b>마스킹</b>입니다.

Keras 모델에 입력 마스크를 도입하는 세 가지 방법이 있습니다:

- `keras.layers.Masking` 레이어를 추가하는 것.
- `mask_zero=True`로 `keras.layers.Embedding` 레이어를 설정하십시오.
- 이 인자를 지원하는 레이어 (예: RNN 레이어)를 호출할 때는 수동으로 `마스크` 인자를 전달하십시오.

## 마스크 생성 레이어: `임베딩` 과 `마스킹`

후드에서 이 레이어들은 마스크텐서 (`(batch, sequence_length)`모양의 2D 텐서)를 만들어 '마스킹'또는 '임베딩' 레이어에서 반환 한 텐서 출력에 연결합니다.

In [0]:
embedding = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)
masked_output = embedding(padded_inputs)

print(masked_output._keras_mask)

In [0]:
masking_layer = layers.Masking()
# 임베딩 차원이 10인 2D 입력을 3D로 확장하여
# 임베딩 조회를 시뮬레이션합니다.
unmasked_embedding = tf.cast(
    tf.tile(tf.expand_dims(padded_inputs, axis=-1), [1, 1, 10]),
    tf.float32)

masked_embedding = masking_layer(unmasked_embedding)
print(masked_embedding._keras_mask)

인쇄 된 결과에서 볼 수 있듯이 마스크는 모양이 batch_size, sequence_length 인 2D 부울 텐서입니다. 여기서 각 개별 False 항목은 처리 중에 해당 시간 단계를 무시해야 함을 나타냅니다.

## 함수형 API 및 순차형 API에서 마스크 전파

함수형 API 또는 순차형 API를 사용할 때 `임베딩` 또는 `마스킹` 계층에서 생성 된 마스크는 이를 사용할 수있는 계층 (예 : RNN 계층)에 대해 네트워크를 통해 전파됩니다. Keras는 입력에 해당하는 마스크를 자동으로 가져 와서 사용 방법을 알고있는 레이어로 전달합니다.

서브 클래싱 된 모델 또는 레이어의 `call` 메소드에서 마스크는 자동으로 전파되지 않으므로 마스크가 필요한 레이어에 수동으로 `마스크` 인자를 전달해야합니다. 자세한 내용은 아래 섹션을 참조하십시오.

예를 들어, 다음 순차적 모델에서 `LSTM` 레이어는 자동으로 마스크를 받습니다. 이는 패딩된 값을 무시한다는 의미입니다:

In [0]:
model = tf.keras.Sequential([
  layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True),
  layers.LSTM(32),
])

다음과 같은 함수형 API 모델의 경우에도 마찬가지입니다:

In [0]:
inputs = tf.keras.Input(shape=(None,), dtype='int32')
x = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)(inputs)
outputs = layers.LSTM(32)(x)

model = tf.keras.Model(inputs, outputs)

## 마스크 텐서를 레이어로 직접 전달

마스크를 처리 할 수 있는 레이어 (예 :`LSTM` 레이어)는 `__call__` 메서드에 `마스크` 인수가 있습니다.

한편 마스크 (예 :`임베딩`)를 생성하는 레이어는 호출 할 수 있는 `compute_mask (input, previous_mask)`메서드를 보이게 합니다.

따라서 다음과 같이 할 수 있습니다:



In [0]:
class MyLayer(layers.Layer):
  
  def __init__(self, **kwargs):
    super(MyLayer, self).__init__(**kwargs)
    self.embedding = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)
    self.lstm = layers.LSTM(32)
    
  def call(self, inputs):
    x = self.embedding(inputs)
    # Note that you could also prepare a `mask` tensor manually.
    # It only needs to be a boolean tensor
    # with the right shape, i.e. (batch_size, timesteps).
    mask = self.embedding.compute_mask(inputs)
    output = self.lstm(x, mask=mask)  # The layer will ignore the masked values
    return output

layer = MyLayer()
x = np.random.random((32, 10)) * 100
x = x.astype('int32')
layer(x)

## Supporting masking in your custom layers

Sometimes you may need to write layers that generate a mask (like `Embedding`), or layers that need to modify the current mask.

For instance, any layer that produces a tensor with a different time dimension than its input, such as a `Concatenate` layer that concatenates on the time dimension, will need to modify the current mask so that downstream layers will be able to properly take masked timesteps into account.

To do this, your layer should implement the `layer.compute_mask()` method, which produces a new mask given the input and the current mask. 

Most layers don't modify the time dimension, so don't need to worry about masking. The default behavior of `compute_mask()` is just pass the current mask through in such cases.

Here is an example of a `TemporalSplit` layer that needs to modify the current mask.

In [0]:
class TemporalSplit(tf.keras.layers.Layer):
  """Split the input tensor into 2 tensors along the time dimension."""

  def call(self, inputs):
    # Expect the input to be 3D and mask to be 2D, split the input tensor into 2
    # subtensors along the time axis (axis 1).
    return tf.split(inputs, 2, axis=1)
    
  def compute_mask(self, inputs, mask=None):
    # Also split the mask into 2 if it presents.
    if mask is None:
      return None
    return tf.split(mask, 2, axis=1)

first_half, second_half = TemporalSplit()(masked_embedding)
print(first_half._keras_mask)
print(second_half._keras_mask)

Here is another example of a `CustomEmbedding` layer that is capable of generating a mask from input values:

In [0]:
class CustomEmbedding(tf.keras.layers.Layer):
  
  def __init__(self, input_dim, output_dim, mask_zero=False, **kwargs):
    super(CustomEmbedding, self).__init__(**kwargs)
    self.input_dim = input_dim
    self.output_dim = output_dim
    self.mask_zero = mask_zero
    
  def build(self, input_shape):
    self.embeddings = self.add_weight(
      shape=(self.input_dim, self.output_dim),
      initializer='random_normal',
      dtype='float32')
    
  def call(self, inputs):
    return tf.nn.embedding_lookup(self.embeddings, inputs)
  
  def compute_mask(self, inputs, mask=None):
    if not self.mask_zero:
      return None
    return tf.not_equal(inputs, 0)
  
  
layer = CustomEmbedding(10, 32, mask_zero=True)
x = np.random.random((3, 10)) * 9
x = x.astype('int32')

y = layer(x)
mask = layer.compute_mask(x)

print(mask)

## Writing layers that need mask information

Some layers are mask *consumers*: they accept a `mask` argument in `call` and use it to determine whether to skip certain time steps.

To write such a layer, you can simply add a `mask=None` argument in your `call` signature. The mask associated with the inputs will be passed to your layer whenever it is available.

```python
class MaskConsumer(tf.keras.layers.Layer):
  
  def call(self, inputs, mask=None):
    ...
```

## Recap

That is all you need to know about masking in Keras. To recap:

- "Masking" is how layers are able to know when to skip / ignore certain timesteps in sequence inputs.
- Some layers are mask-generators: `Embedding` can generate a mask from input values (if `mask_zero=True`), and so can the `Masking` layer.
- Some layers are mask-consumers: they expose a `mask` argument in their `__call__` method. This is the case for RNN layers.
- In the Functional API and Sequential API, mask information is propagated automatically.
- When writing subclassed models or when using layers in a standalone way, pass the `mask` arguments to layers manually.
- You can easily write layers that modify the current mask, that generate a new mask, or that consume the mask associated with the inputs.
